# Chapter 5

This file is used for the computations used in the LDM book, Chapter 5.

In [1]:
%matplotlib notebook

import numpy as np
import numpy.random as rnd
import numpy.linalg as la
import numpy.matlib as mat
import scipy.sparse as sp
import matplotlib.pyplot as plt

import models
import domains as env
import solutions as sol

import time
import math

plt.rc('text', usetex = True)
plt.rc('font', family = 'serif', serif = 'Computer Modern Roman', size = 18)

def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

### 3-state example

In [2]:
M = env.three_states()
pol1 = sol.Policy(M, weights=np.array([[1, 0],[1, 0],[1, 0]]))
pol2 = sol.Policy(M, weights=np.array([[0, 1],[0, 1],[0, 1]]))

Q1 = sol.value_iteration(M, pol1, cost=False)
Q2 = sol.value_iteration(M, pol2, cost=False)

print('Policy 1:')
print(pol1)
print('Cost-to-go:')
print(pol1.expectation(func=Q1))

print('\nPolicy 2:')
print(pol2)
print('Cost-to-go:')
print(pol2.expectation(func=Q2))

Policy 1:
0 -> a
A -> a
B -> a

Cost-to-go:
[  1.   0. 100.]

Policy 2:
0 -> b
A -> b
B -> b

Cost-to-go:
[ 99.5   0.  100. ]


### Hiring example (N = 2)

In [3]:
M = env.hiring(2)

nS = M.n_states()
nA = M.n_actions()

# VI cycle

Q = np.zeros([nS, nA])

EPS = 0.5 * 1e-8 / M.gamma() * (1 - M.gamma())
Err = 1
it = 0

print('\n --  VI  -- ')

while Err > EPS:
    
    print('Q('+str(it)+'):')
    print(Q)

    # Buffer for stopping condition
    Qnew = np.zeros([nS, nA])

    # VI update
    for a in range(nA):
        Qnew[:, a] = M.c(a) + M.gamma() * np.dot(M.p(a = a), Q.min(axis = 1))

    # Stopping condition
    Err = la.norm(Qnew - Q, ord = 2);

    Q = Qnew
    it = it + 1
    
print('Q('+str(it)+'):')
print(Q)

# PI cycle

print('\n --  PI  -- ')

pol = sol.Uniform(M)

quit = False
it = 0

while not quit:
    
    print('pi('+str(it)+'):')
    print(pol.probability())

    Q = sol.value_iteration(M, pol, cost=False)
    
    print('J('+str(it)+'):')
    print(pol.expectation(func=Q))

    polnew = sol.Greedy(M, Q)

    it = it + 1        
    quit = pol == polnew
    pol = polnew
    
print('pi('+str(it)+'):')
print(pol.probability())

Qvi, foo = sol.value_iteration(M)
pvi = sol.Greedy(M, Qvi)

ppi, foo = sol.policy_iteration(M)
Qpi = ppi.evaluate(M, cost=False)

print('\n= Hiring scenario (N = 2) =\n')
print('-- Value iteration --')
print('Q-function:')
print(Qvi)
print('Policy:')
print(pvi)

print('\n-- Policy iteration --')
print('Q-function:')
print(Qpi)
print('Policy:')
print(ppi)


 --  VI  -- 
Q(0):
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
Q(1):
[[ 0.5  0. ]
 [ 0.   0. ]
 [ 1.   1. ]
 [ 0.   0. ]]
Q(2):
[[ 0.5    0.475]
 [ 0.     0.   ]
 [ 1.     1.   ]
 [ 0.     0.   ]]
Q(3):
[[ 0.5    0.475]
 [ 0.     0.   ]
 [ 1.     1.   ]
 [ 0.     0.   ]]

 --  PI  -- 
pi(0):
[[ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]]
J(0):
[ 0.4875  0.      1.      0.    ]
pi(1):
[[ 0.   1. ]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]]
J(1):
[ 0.475  0.     1.     0.   ]
pi(2):
[[ 0.   1. ]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]]

= Hiring scenario (N = 2) =

-- Value iteration --
Q-function:
[[ 0.5    0.475]
 [ 0.     0.   ]
 [ 1.     1.   ]
 [ 0.     0.   ]]
Policy:
(B,1) -> ~H
(B,2) -> {H, ~H}
(~B,2) -> {H, ~H}
H -> {H, ~H}


-- Policy iteration --
Q-function:
[[ 0.5    0.475]
 [ 0.     0.   ]
 [ 1.     1.   ]
 [ 0.     0.   ]]
Policy:
(B,1) -> ~H
(B,2) -> {H, ~H}
(~B,2) -> {H, ~H}
H -> {H, ~H}



### Hiring example (N = 3)

In [4]:
M = env.hiring(3)

Qvi, foo = sol.value_iteration(M)
pvi = sol.Greedy(M, Qvi)

ppi, foo = sol.policy_iteration(M)
Qpi = ppi.evaluate(M, cost=False)

print('\n= Hiring scenario (N = 3) =\n')
print('-- Value iteration --')
print('Q-function:')
print(Qvi)
print('Policy:')
print(pvi)

print('\n-- Policy iteration --')
print('Q-function:')
print(Qpi)
print('Policy:')
print(ppi)


= Hiring scenario (N = 3) =

-- Value iteration --
Q-function:
[[ 0.66666667  0.45916667]
 [ 0.33333333  0.63333333]
 [ 1.          0.63333333]
 [ 0.          0.        ]
 [ 1.          1.        ]
 [ 0.          0.        ]]
Policy:
(B,1) -> ~H
(B,2) -> H
(~B,2) -> ~H
(B,3) -> {H, ~H}
(~B,3) -> {H, ~H}
H -> {H, ~H}


-- Policy iteration --
Q-function:
[[ 0.66666667  0.45916667]
 [ 0.33333333  0.63333333]
 [ 1.          0.63333333]
 [ 0.          0.        ]
 [ 1.          1.        ]
 [ 0.          0.        ]]
Policy:
(B,1) -> ~H
(B,2) -> H
(~B,2) -> ~H
(B,3) -> {H, ~H}
(~B,3) -> {H, ~H}
H -> {H, ~H}



### Hiring example (N = 5, ..., 100)

In [5]:
VALUES = (2, 3, 5, 10, 20, 50, 100)
iter_vi = ()
iter_pi = ()

for N in log_progress(VALUES, every = 1):
    M = env.hiring(N)

    Qvi, aux = sol.value_iteration(M)
    iter_vi += (aux,)

    ppi, aux = sol.policy_iteration(M)
    iter_pi += (aux,)
    
iter_vi = np.array(iter_vi)
iter_pi = np.array(iter_pi)

Plot the values from the previous runs

In [6]:
plt.figure()
plt.plot(VALUES, iter_vi, 'k-', linewidth = 3, label = 'VI')
plt.plot(VALUES, iter_pi, 'k--', linewidth = 3, label = 'PI')
plt.xlabel('$N$')
plt.ylabel('N. iterations')
plt.legend(loc = 'best')
plt.show()
plt.savefig('hiring-iterations.pdf')

<IPython.core.display.Javascript object>

### Epidemy example

This example compares the time/iterations/error decay in VI and PI.

In [7]:
M = env.epidemic(10)

pol, i = sol.policy_iteration(M)
print('Policy:')
print(pol)
print('Q-function:')
print(pol.evaluate(M, cost=False))
print('Iterations:', i)

Policy:
0 -> {(10, 0), (10, 10), (10, 50), (10, 100)}
1 -> (10, 50)
2 -> (10, 50)
3 -> (10, 50)
4 -> (10, 50)
5 -> (10, 50)
6 -> (10, 50)
7 -> (10, 50)
8 -> (10, 50)
9 -> (10, 50)
10 -> (0, 10)

Q-function:
[[ 0.12970276  0.12970276  0.12970276  0.12970276  0.12813621  0.12813621
   0.12813621  0.12813621  0.13600849  0.13600849  0.13600849  0.13600849
   0.17212637  0.17212637  0.17212637  0.17212637]
 [ 0.21837161  0.21654187  0.21374456  0.21458839  0.21669337  0.21486363
   0.21206631  0.21291015  0.22434256  0.22251282  0.2197155   0.22055934
   0.26038573  0.25855599  0.25575868  0.25660251]
 [ 0.32248023  0.31848844  0.31218077  0.31361263  0.32066176  0.31666997
   0.3103623   0.31179416  0.32803086  0.32403907  0.3177314   0.31916326
   0.36398024  0.35998845  0.35368078  0.35511265]
 [ 0.44685647  0.44043052  0.4299938   0.43178049  0.44492694  0.43850099
   0.42806426  0.42985096  0.45207421  0.44564825  0.43521153  0.43699823
   0.48794931  0.48152336  0.47108664  0.4728733

In [8]:
VALUES = (1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 100)
NITER = 50

# First, time and iterations

time_vi = np.zeros(len(VALUES))
iter_vi = np.zeros(len(VALUES))

time_pi = np.zeros(len(VALUES))
iter_pi = np.zeros(len(VALUES))

n_index = 0

for N in log_progress(VALUES, every = 1):
    M = env.epidemic(N)
    
    tvi = 0
    tpi = 0
    ivi = 0
    ipi = 0

    for j in range(NITER):
        start = time.time()
        Qvi, i = sol.value_iteration(M)
        tvi = tvi + time.time() - start
        ivi = ivi + i
    
        start = time.time()
        pol, i = sol.policy_iteration(M)
        tpi = tpi + time.time() - start
        ipi = ipi + i

    time_vi[n_index] = tvi / NITER
    iter_vi[n_index] = ivi / NITER

    time_pi[n_index] = tpi / NITER
    iter_pi[n_index] = ipi / NITER
    
    n_index = n_index + 1

KeyboardInterrupt: 

Now error decay.

In [ ]:
EMAX = 1e-10

# Now error decay

Qopt = pol.evaluate(M, cost=False)

err_vi = []

Q = np.zeros((M.n_states(), M.n_actions()))
err = 1

while err > EMAX:
    
    for a in range(M.n_actions()):
        Q[:, a] = M.c(a) + M.gamma() * np.dot(M.p(a = a), Q.min(axis = 1))

    err = la.norm(Qopt - Q, ord = 2)
    err_vi = err_vi + [err]
    
err_vi = np.array(err_vi)

Plotting...

In [ ]:
print('VI -- time per iteration:', time_vi[-1]/iter_vi[-1])
print('PI -- time per iteration:', time_pi[-1]/iter_pi[-1])
      
fig = plt.figure()

plt.subplot(211)
plt.plot(VALUES, iter_vi, 'k-', linewidth = 3, label = 'VI')
plt.plot(VALUES, iter_pi, 'k--', linewidth = 3, label = 'PI')
plt.ylabel('N. of iterations')
plt.legend(loc = 'best', fontsize='x-small')
plt.locator_params(axis='y', nbins = 5)

plt.subplot(212)
plt.plot(VALUES, time_vi, 'k-', linewidth = 3, label = 'VI')
plt.plot(VALUES, time_pi, 'k--', linewidth = 3, label = 'PI')
plt.xlabel('$N$')
plt.ylabel('Computation time (sec.)')
plt.legend(loc = 'best', fontsize='x-small')
plt.locator_params(axis='y', nbins = 5)

plt.show()

plt.savefig('epidemic-time-iter.pdf')

plt.figure()
plt.plot(err_vi[:500], 'k', linewidth = 3, label = 'VI')
plt.xlabel('N. iterations')
plt.ylabel('Error ($\|Q^{(k)}-Q^*\|_2$)')
plt.show()
plt.savefig('epidemic-error.pdf')

### Fault detection example

MDP solution (for discussion purposes).

In [ ]:
N = 3
p = np.array(range(1, N + 1)) / (N + 2)
t = np.array(range(N, 0, -1)) / (N + 2)
M = env.fault(N, p, t)    
pol, i = sol.policy_iteration(M)
print('N. iterations:', i)
print('Policy:')
print(pol)

Illustration of the growth in compexity of the MDP as the dimension of the problem grows.

In [ ]:
VALUES = (2, 3, 4, 5, 6, 7, 8)
NITER = 50

# First, time and iterations

time_vi = np.zeros(len(VALUES))
iter_vi = np.zeros(len(VALUES))

time_pi = np.zeros(len(VALUES))
iter_pi = np.zeros(len(VALUES))

size = np.zeros(len(VALUES))

n_index = 0

for N in log_progress(VALUES, every = 1):
    p = np.array(range(1, N + 1)) / (N + 2)
    t = np.array(range(N, 0, -1)) / (N + 2)
    M = env.fault(N, p, t)
        
    size[n_index] = len(M)
    
    tvi = 0
    tpi = 0
    ivi = 0
    ipi = 0

    for j in log_progress(range(NITER), every = 1):
        start = time.time()
        Qvi, i = sol.value_iteration(M)
        tvi = tvi + time.time() - start
        ivi = ivi + i
    
        start = time.time()
        pol, i = sol.policy_iteration(M)
        tpi = tpi + time.time() - start
        ipi = ipi + i

    time_vi[n_index] = tvi / NITER
    iter_vi[n_index] = ivi / NITER

    time_pi[n_index] = tpi / NITER
    iter_pi[n_index] = ipi / NITER
    
    n_index = n_index + 1

And the plot...

In [ ]:
print('VI -- time per iteration:', time_vi[-1]/iter_vi[-1])
print('PI -- time per iteration:', time_pi[-1]/iter_pi[-1])
      
fig = plt.figure()

plt.subplot(211)
plt.plot(size, iter_vi, 'k-', linewidth = 3, label = 'VI')
plt.plot(size, iter_pi, 'k--', linewidth = 3, label = 'PI')
plt.ylabel('N. of iterations')
plt.axis([0, 10000, 0, 10])
plt.legend(loc = 'best', fontsize='x-small')
plt.locator_params(axis='y', nbins=5)

plt.subplot(212)
plt.plot(size, time_vi, 'k-', linewidth = 3, label = 'VI')
plt.plot(size, time_pi, 'k--', linewidth = 3, label = 'PI')
plt.xlabel(r'MDP size ($|\mathcal{X}|\times|\mathcal{A}|$)')
plt.ylabel('Computation time (sec.)')
plt.legend(loc = 'best', fontsize='x-small')
plt.axis([0, 10000, 0, 2])
plt.locator_params(axis='y', nbins=4)

plt.show()
plt.savefig('fault-time-iter.pdf')

### Bus example

In [ ]:
M = env.bus(2)
Q,i = sol.linear_programming(M)
print('Iterations:', i)
print('Q:')
print(Q)
print('Policy:')
print(sol.Greedy(M, Q))

In [ ]:
VALUES = range(2,7)
NITER = 30

time_vi = np.zeros(len(VALUES))
iter_vi = np.zeros(len(VALUES))

time_pi = np.zeros(len(VALUES))
iter_pi = np.zeros(len(VALUES))

time_lp = np.zeros(len(VALUES))
iter_lp = np.zeros(len(VALUES))

size = np.zeros(len(VALUES))

n_index = 0

for N in log_progress(VALUES, every = 1):
    p = np.array(range(1, N + 1)) / (N + 2)
    t = np.array(range(N, 0, -1)) / (N + 2)
    M = env.fault(N, p, t)
    
    size[n_index] = len(M)
    
    tvi = 0
    tpi = 0
    tlp = 0
    ivi = 0
    ipi = 0
    ilp = 0

    for j in log_progress(range(NITER), every = 1):
        start = time.time()
        Qvi, i = sol.value_iteration(M)
        tvi = tvi + time.time() - start
        ivi = ivi + i
    
        start = time.time()
        pol, i = sol.policy_iteration(M)
        tpi = tpi + time.time() - start
        ipi = ipi + i

        start = time.time()
        Qlp, i = sol.linear_programming(M)
        tlp = tlp + time.time() - start
        ilp = ilp + i

    time_vi[n_index] = tvi / NITER
    iter_vi[n_index] = ivi / NITER

    time_pi[n_index] = tpi / NITER
    iter_pi[n_index] = ipi / NITER
    
    time_lp[n_index] = tlp / NITER
    iter_lp[n_index] = ilp / NITER

    n_index = n_index + 1

And we plot...

In [ ]:
print('VI -- time per iteration:', time_vi[-1]/iter_vi[-1])
print('PI -- time per iteration:', time_pi[-1]/iter_pi[-1])
print('LP -- time per iteration:', time_lp[-1]/iter_lp[-1])
      
fig = plt.figure()

#plt.subplot(211)
#plt.semilogy(size, iter_vi, 'k-', linewidth = 3, label = 'VI')
#plt.semilogy(size, iter_pi, 'k--', linewidth = 3, label = 'PI')
#plt.semilogy(size, iter_lp, 'k:', linewidth = 3, label = 'LP')
#plt.ylabel('N. of iterations')
#plt.legend(loc = 'best', fontsize='x-small')

#plt.subplot(212)
plt.semilogy(size, time_vi, 'k-', linewidth = 3, label = 'VI')
plt.semilogy(size, time_pi, 'k--', linewidth = 3, label = 'PI')
plt.semilogy(size, time_lp, 'k:', linewidth = 3, label = 'LP')
plt.xlabel(r'MDP size ($|\mathcal{X}|\times|\mathcal{A}|$)')
plt.ylabel('Computation time (sec.)')
plt.legend(loc = 'best', fontsize='x-small')
plt.axis([0, 1500, 1e-4, 1e1])

plt.show()
plt.savefig('bus-time.pdf')

### Divergent example

In [ ]:
def state_map(x = None):

    f = np.array([[1, 0, 1, 1, 0, 0],
                  [1, 0, 0, 0, 1, 1],
                  [0, 1, 0, 1, 1, 0],
                  [0, 1, 1, 0, 0, 1]])               

    if x != None:
        f = f[:, x - 1]

    return f

M = env.divergent()

print(M)

EPS = 1e-8
NITER = 21

W = np.zeros(NITER)
w = np.ones((4, 2))

# Initialization

iter = 0

# VI cycle

while iter < NITER:
    
    W[iter] = la.norm(w)
    
    Q = np.dot(state_map().transpose(), w)
    w[:, 0] = np.dot(state_map(), (M.c(0) + M.gamma() * np.dot(M.p(a = 0), Q.min(axis = 1))))
    w[:, 1] = np.dot(state_map(), (M.c(1) + M.gamma() * np.dot(M.p(a = 1), Q.min(axis = 1))))   
       
    iter = iter + 1

fig = plt.figure()
plt.semilogy(W, 'k-', linewidth = 3, label = 'norm')
plt.xlabel('N. iterations')
plt.ylabel('\|\mathbf{W}\|_2')

plt.show()
plt.savefig('divergent-result.pdf')

### Mountain car problem

Illustrating the use of function approximation.

In [ ]:
PMIN = -1.2
PMAX = 0.6

VMIN = -0.07
VMAX = 0.07
    

def carhill(step = 200):    
    # -- Simulation of the car (forward)
    
    g = 0.0025
    
    nS = step ** 2
    
    states = ()
    actions = ('L', 'N', 'R')
    
    Pl = sp.dok_matrix((nS, nS))
    Pn = sp.dok_matrix((nS, nS))
    Pr = sp.dok_matrix((nS, nS))
    
    C = np.ones(nS)    
        
    dv = (VMAX - VMIN) / (step - 1)
    dp = (PMAX - PMIN) / (step - 1)
    
    for i in log_progress(range(step), every = 1):
        v = round(VMIN + i * dv, 4)

        for j in range(step):
            p = round(PMIN + j * dp, 4)
            
            # Initial index

            x  = i * step + j
            
            # Add state            

            states = states + ((p, v),)
            
            # Compute next velocity
            
            vl = v - 0.001 - g * math.cos(3 * p)
            vn = v - g * math.cos(3 * p)
            vr = v + 0.001 - g * math.cos(3 * p)
            
            # Set bounds
            
            vl = round(max(min(vl, VMAX), VMIN), 4)
            vn = round(max(min(vn, VMAX), VMIN), 4)
            vr = round(max(min(vr, VMAX), VMIN), 4)
            
            # Compute next position
            
            if p == PMAX:
                pl = p
                pn = p
                pr = p
                C[x] = 0
            else:
                pl = p + vl
                pn = p + vn
                pr = p + vr
            
            # Set bounds - velocity is clipped as in Singh & Sutton, ML, 96
            
            paux = max(min(pl, PMAX), PMIN)
            if paux != pl:
                vl = 0
            pl = round(paux, 4)

            paux = max(min(pn, PMAX), PMIN)
            if paux != pn:
                vn = 0
            pn = round(paux, 4)

            paux = max(min(pr, PMAX), PMIN)
            if paux != pr:
                vr = 0
            pr = round(paux, 4)
            
            # Discretize
            
            vl_index = round((vl - VMIN) / dv)
            vn_index = round((vn - VMIN) / dv)
            vr_index = round((vr - VMIN) / dv)
            
            pl_index = round((pl - PMIN) / dp)
            pn_index = round((pn - PMIN) / dp)
            pr_index = round((pr - PMIN) / dp)
            
            # Compute indices
            
            yl = vl_index * step + pl_index
            yn = vn_index * step + pn_index
            yr = vr_index * step + pr_index
            
            # Fill transitions
            
            Pl[x, yl] = 1
            Pn[x, yn] = 1
            Pr[x, yr] = 1
            
    P = [Pl, Pn, Pr]
        
    g = 0.95
    
    return {'states' : states, 'actions' : actions, 'P' : P, 'C' : C, 'gamma' : g}

# Parameters 

STEP = 300
EPS = 1e-8

print('Building MDP...')

M = carhill(STEP)

nS = len(M['states'])
nA = len(M['actions'])

Q = np.zeros([nS, nA])

Err = 1
iter = 0

# VI cycle

print('Running VI', end = '', flush = True)

while Err > EPS:
    
    if iter % 20 == 0:
        print('.', end = '', flush = True)
    
    # Buffer for stopping condition
    Qnew = np.zeros([nS, nA])
    
    # VI update
    J = Q.min(axis = 1)
    
    Qnew[:, 0] = M['C'] + M['gamma'] * M['P'][0].dot(J)
    Qnew[:, 1] = M['C'] + M['gamma'] * M['P'][1].dot(J)
    Qnew[:, 2] = M['C'] + M['gamma'] * M['P'][2].dot(J)
        
    # Stopping condition
    Err = la.norm(Qnew - Q, ord = 2);
    
    Q = Qnew
    iter = iter + 1
    
# Computation statistics
print(' Done.\nFinished in', iter, 'iterations.', flush = True)

Plotting...

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

X = []
Y = []

for state in M['states']:
    X = X + [state[0]]
    Y = Y + [state[1]]

X = np.array(X)
Y = np.array(Y)
Z = Q.min(axis = 1)

X = X.reshape((STEP, STEP))
Y = Y.reshape((STEP, STEP))
Z = Z.reshape((STEP, STEP))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot a basic wireframe.
ax.plot_wireframe(X, Y, Z, rstride = max(5, STEP // 50), cstride = max(5, STEP // 50), color = 'black')
plt.show()

Fixing the plot...

In [ ]:
ax.view_init(azim=120, elev=40)
plt.axis([-1.2, 0.6, -0.07, 0.07])
plt.locator_params(axis='y', nbins=4)
ax.set_xlabel('Position', labelpad = 15)
ax.set_ylabel('Velocity', labelpad = 15)
ax.set_zlabel('Cost-to-go ($J^*$)', labelpad = 10)

plt.savefig('mountain-car-optimal.pdf')

Comparing non-averager with averager approaches. Building non-averager (RBF)...

In [ ]:
N_POINTS = 60

def state_map(M, x = None, step = 200):
    
    dV = (VMAX - VMIN) / (N_POINTS - 1)
    dP = (PMAX - PMIN) / (N_POINTS - 1)

    dv = (VMAX - VMIN) / (step - 1)
    dp = (PMAX - PMIN) / (step - 1)
    
    p = mat.repmat(np.arange(0, PMAX - PMIN + dp, dp).reshape((1, step)), 1, step).reshape(step ** 2)    
    v = mat.repmat(np.arange(0, VMAX - VMIN + dv, dv).reshape((step, 1)), 1, step).reshape(step ** 2)
    
    print('p in [' + str(p[0]) + ', ' + str(p[-1]) + ']; v in [' + str(v[0]) + ', ' + str(v[-1]) + ']')
    
    f = np.zeros((N_POINTS ** 2, step ** 2))
    
    sv = (0.5 * dV) ** 2
    sp = (3.0 * dP) ** 2
    
    for idx in log_progress(range(N_POINTS ** 2), every = 1):
        idx_p = idx % N_POINTS
        idx_v = idx // N_POINTS

        fp = abs(p - idx_p * dP)
        fv = abs(v - idx_v * dV)
        
        f[idx, :] = np.exp(- fp ** 2 / sp - fv ** 2 / sv)        
        
    if x != None:
        ip = round((x[0] - PMIN) / dp)
        iv = round((x[1] - VMIN) / dv)
        x = iv * step + ip
        f = f[:, x]
        z = f.sum()
        return f / z
    
    # Normalizing... 
    print('f shape:', f.shape)
    Z = np.reshape(f.sum(axis = 0), (1, len(M['states'])))
    return f / Z

# Compute feature set and track time

print('Computing feature set...')
b = time.time()
f_rbf = state_map(M, step = STEP)
t = time.time() - b
print('Done in', round(t), 'seconds.')

Running VI...

In [ ]:
nS = len(M['states'])
nA = len(M['actions'])
nF = f_rbf.shape[0]
W = np.zeros((nF, nA))

Err = 1
iter = 0

# VI cycle

print('Running VI', end = '', flush = True)

while Err > EPS:
    
    if iter % 20 == 0:
        print('.', end = '', flush = True)
    
    # Buffer for stopping condition
    Wnew = np.zeros((nF, nA))
    
    # VI update
    Q = np.dot(f_rbf.transpose(),W)
    J = Q.min(axis = 1)
    
    Wnew[:, 0] = np.dot(f_rbf,M['C'] + M['gamma'] * M['P'][0].dot(J))
    Wnew[:, 1] = np.dot(f_rbf,M['C'] + M['gamma'] * M['P'][1].dot(J))
    Wnew[:, 2] = np.dot(f_rbf,M['C'] + M['gamma'] * M['P'][2].dot(J))
        
    # Stopping condition
    Err = la.norm(Wnew - W, ord = 2);
    
    W = Wnew
    iter = iter + 1
    
    if la.norm(W) > 1e10:
        break
    
# Computation statistics
print(' Done.\nFinished in', iter, 'iterations.', flush = True)
Q_rbf = np.dot(f_rbf.transpose(), W)

Plotting...

In [ ]:
Z_rbf = Q_rbf.min(axis = 1)
Z_rbf = Z_rbf.reshape((STEP, STEP))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot a basic wireframe.
ax.plot_wireframe(X, Y, Z_rbf, rstride = max(5, STEP // 30), cstride = max(5, STEP // 30), color = 'black')
plt.show()

Fixing the plot...

In [ ]:
ax.view_init(azim=120, elev=40)
plt.axis([-1.2, 0.6, -0.07, 0.07])
plt.locator_params(axis='z', nbins=4)
plt.locator_params(axis='y', nbins=4)
ax.set_xlabel('Position', labelpad = 15)
ax.set_ylabel('Velocity', labelpad = 15)
ax.set_zlabel('$J^*$', labelpad = 10)

plt.savefig('mountain-car-rbf.pdf')

Building averager (1-NN)...

In [ ]:
nF = f_rbf.shape[0]

print('Computing feature set...', flush = True)
b = time.time()
f_knn = f_rbf.T
aux = f_rbf.max(axis = 1).reshape((nF, 1))
f_inv = sp.dok_matrix((f_rbf == aux).astype(int))
t = time.time() - b
print('Done in', round(t), 'seconds.', flush = True)

Running VI...

In [ ]:
nS = len(M['states'])
nA = len(M['actions'])
W = np.zeros((nF, nA))

Err = 1
iter = 0
np.set_printoptions(precision = 3, linewidth = 120)
print('Computing C... ', end = '', flush = True)
C = f_inv.dot(M['C'])
print('Done.\nComputing P... ', end = '', flush = True)
P=dict() 
aux = f_inv.dot(M['P'][0])
P[0] = aux.dot(f_knn)
print('Done with 0... ', end = '', flush = True)
aux = f_inv.dot(M['P'][1])
P[1] = aux.dot(f_knn)
print('Done with 1... ', end = '', flush = True)
aux = f_inv.dot(M['P'][2])
P[2] = aux.dot(f_knn)
print('Done with 2.')

print('C:', C.shape, '- min:', C.min(), '- max:', C.max())
print('P[0]:', P[0].shape, '- sum:', P[0].sum(axis=1).min())
print('P[1]:', P[1].shape, '- sum:', P[1].sum(axis=1).min())
print('P[2]:', P[2].shape, '- sum:', P[2].sum(axis=1).min())

# VI cycle

print('Running VI', end = '', flush = True)

while Err > EPS:
    
    if iter % 20 == 0:
        print('.', end = '', flush = True)
    
    # Buffer for stopping condition
    Wnew = np.zeros((nF, nA))
    
    # VI update    
    J = W.min(axis = 1)
    
    Wnew[:, 0] = C + M['gamma'] * P[0].dot(J)
    Wnew[:, 1] = C + M['gamma'] * P[1].dot(J)
    Wnew[:, 2] = C + M['gamma'] * P[2].dot(J)
        
    # Stopping condition
    Err = la.norm(Wnew - W, ord = 2);
    
    W = Wnew
    iter = iter + 1
    
# Computation statistics
print(' Done.\nFinished in', iter, 'iterations.', flush = True)
Q_knn = f_knn.dot(W)

Plotting...

In [ ]:
Z_knn = Q_knn.min(axis = 1)
print('Z shape:', Z_knn.shape, '- step:', STEP)
print('Qmin:', Q_knn.min(axis=0).min(), '- Qmax:', Q_knn.max(axis=0).max())
Z_knn = Z_knn.reshape((STEP, STEP))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot a basic wireframe.
ax.plot_wireframe(X, Y, Z_knn, rstride = max(5, STEP // 50), cstride = max(5, STEP // 50), color = 'black')
plt.show()

Fixing the plot...

In [ ]:
ax.view_init(azim=120, elev=40)
ax.set_xlim3d(-1.2, 0.6)
ax.set_ylim3d(-0.07,0.07)
ax.set_zlim3d(0,20)
plt.locator_params(axis='z', nbins=5)
plt.locator_params(axis='y', nbins=4)
ax.set_xlabel('Position', labelpad = 15)
ax.set_ylabel('Velocity', labelpad = 15)
ax.set_zlabel('$J^*$', labelpad = 10)

plt.savefig('mountain-car-knn.pdf')